**Exploring Hacker News Posts**

In this project, we'll compare two different types of posts from Hacker News, a popular site where technology related stories (or 'posts') are voted and commented upon. The two types of posts we'll explore begin with either Ask HN or Show HN.

Users submit Ask HN posts to ask the Hacker News community a specific question, such as "What is the best online course you've ever taken?" Likewise, users submit Show HN posts to show the Hacker News community a project, product, or just generally something interesting.

We'll specifically compare these two types of posts to determine the following:

1.Do Ask HN or Show HN receive more comments on average?

2.Do posts created at a certain time receive more comments on average?

It should be noted that the data set we're working with was reduced from almost 300,000 rows to approximately 20,000 rows by removing all submissions that did not receive any comments, and then randomly sampling from the remaining submissions.

**Introduction**


First, we'll read in the data and remove the headers.

In [1]:
from csv import reader
opened_file = open('hacker_news.csv')
read_file = reader(opened_file)
hn = list(read_file)

headers = hn[0]
hn = hn[1:]
print(headers)
print('\n')
print(hn[:5])

['id', 'title', 'url', 'num_points', 'num_comments', 'author', 'created_at']


[['12224879', 'Interactive Dynamic Video', 'http://www.interactivedynamicvideo.com/', '386', '52', 'ne0phyte', '8/4/2016 11:52'], ['10975351', 'How to Use Open Source and Shut the Fuck Up at the Same Time', 'http://hueniverse.com/2016/01/26/how-to-use-open-source-and-shut-the-fuck-up-at-the-same-time/', '39', '10', 'josep2', '1/26/2016 19:30'], ['11964716', "Florida DJs May Face Felony for April Fools' Water Joke", 'http://www.thewire.com/entertainment/2013/04/florida-djs-april-fools-water-joke/63798/', '2', '1', 'vezycash', '6/23/2016 22:20'], ['11919867', 'Technology ventures: From Idea to Enterprise', 'https://www.amazon.com/Technology-Ventures-Enterprise-Thomas-Byers/dp/0073523429', '3', '1', 'hswarna', '6/17/2016 0:01'], ['10301696', 'Note by Note: The Making of Steinway L1037 (2007)', 'http://www.nytimes.com/2007/11/07/movies/07stein.html?_r=0', '8', '2', 'walterbell', '9/30/2015 4:12']]


**Extracting Ask HN and Show HN Posts¶**


We only need to analyze Ask HN or Show HN, so this data should be isolated. 

In [2]:
ask_posts = []
show_posts =[]
other_posts = []

for post in hn:
    title = post[1]
    if title.lower().startswith("ask hn"):
        ask_posts.append(post)
    elif title.lower().startswith("show hn"):
        show_posts.append(post)
    else:
        other_posts.append(post)
        
print('Number of posts:', len(ask_posts))
print('Number of posts:', len(show_posts))
print('Number of posts:', len(other_posts))


Number of posts: 1744
Number of posts: 1162
Number of posts: 17194


**Calculating the Average Number of Comments for Ask HN and Show HN Posts**

Let's find out which type of posts receive more comments on average.

In [3]:
total_ask_comments = 0

for post in ask_posts:
    num_comments = int(post[4])
    total_ask_comments += num_comments
    avg_ask_comments = total_ask_comments/len(ask_posts)
    
print("Average number of comment:", avg_ask_comments)

Average number of comment: 14.038417431192661


In [4]:
total_show_comments = 0
    
for post in show_posts:
    num_comments = int(post[4])
    total_show_comments += num_comments
    avg_show_comments = total_show_comments/len(show_posts)
    
print("Average number of comment:", avg_show_comments)

Average number of comment: 10.31669535283993


On average, ask posts in our sample receive approximately 14 comments, whereas show posts receive approximately 10. Since ask posts are more likely to receive comments, we'll focus our remaining analysis just on these posts.

**Finding the Amount of Ask Posts and Comments by Hour Created**

Next, we'll determine if ask posts created at a certain time are more likely to attract comments. We'll use the following steps to perform this analysis:

1. Calculate the amount of ask posts created in each hour of the day, along with the number of comments received.
2. Calculate the average number of comments ask posts receive by hour created.

In [5]:
import datetime as dt

result_list = []

for post in ask_posts:
    result_list.append(
        [post[6], int(post[4])]
    )

comments_by_hour = {}
counts_by_hour = {}
date_format = "%m/%d/%Y %H:%M"

for row in result_list:
    hour = row[0]
    comment = row[1]
    time = dt.datetime.strptime(hour, date_format).strftime("%H")
    if time in counts_by_hour:
        comments_by_hour[time] += comment
        counts_by_hour[time] += 1
    else:
        comments_by_hour[time] = comment
        counts_by_hour[time] = 1
        
print(counts_by_hour)
print('\n')
print(comments_by_hour)


{'02': 58, '22': 71, '19': 110, '12': 73, '00': 55, '05': 46, '17': 100, '21': 109, '14': 107, '07': 34, '08': 48, '16': 108, '01': 60, '10': 59, '06': 44, '04': 47, '23': 68, '20': 80, '15': 116, '11': 58, '09': 45, '13': 85, '18': 109, '03': 54}


{'02': 1381, '22': 479, '19': 1188, '12': 687, '00': 447, '05': 464, '17': 1146, '21': 1745, '14': 1416, '07': 267, '08': 492, '16': 1814, '01': 683, '10': 793, '06': 397, '04': 337, '23': 543, '20': 1722, '15': 4477, '11': 641, '09': 251, '13': 1253, '18': 1439, '03': 421}


Next, we'll use these two dictionaries to calculate the average number of comments for posts created during each hour of the day.

**Calculating the Average Number of Comments for Ask HN Posts by Hour**

In [8]:
avg_by_hour = []

for hr in comments_by_hour:
    avg_by_hour.append([hr, comments_by_hour[hr] / counts_by_hour[hr]])

avg_by_hour

[['02', 23.810344827586206],
 ['22', 6.746478873239437],
 ['19', 10.8],
 ['12', 9.41095890410959],
 ['00', 8.127272727272727],
 ['05', 10.08695652173913],
 ['17', 11.46],
 ['21', 16.009174311926607],
 ['14', 13.233644859813085],
 ['07', 7.852941176470588],
 ['08', 10.25],
 ['16', 16.796296296296298],
 ['01', 11.383333333333333],
 ['10', 13.440677966101696],
 ['06', 9.022727272727273],
 ['04', 7.170212765957447],
 ['23', 7.985294117647059],
 ['20', 21.525],
 ['15', 38.5948275862069],
 ['11', 11.051724137931034],
 ['09', 5.5777777777777775],
 ['13', 14.741176470588234],
 ['18', 13.20183486238532],
 ['03', 7.796296296296297]]

**Sorting and Printing Values from a List of Lists**

In [9]:
swap_avg_by_hour = []

for row in avg_by_hour:
    swap_avg_by_hour.append([row[1], row[0]])
    
print(swap_avg_by_hour)

sorted_swap = sorted(swap_avg_by_hour, reverse=True)

sorted_swap

[[23.810344827586206, '02'], [6.746478873239437, '22'], [10.8, '19'], [9.41095890410959, '12'], [8.127272727272727, '00'], [10.08695652173913, '05'], [11.46, '17'], [16.009174311926607, '21'], [13.233644859813085, '14'], [7.852941176470588, '07'], [10.25, '08'], [16.796296296296298, '16'], [11.383333333333333, '01'], [13.440677966101696, '10'], [9.022727272727273, '06'], [7.170212765957447, '04'], [7.985294117647059, '23'], [21.525, '20'], [38.5948275862069, '15'], [11.051724137931034, '11'], [5.5777777777777775, '09'], [14.741176470588234, '13'], [13.20183486238532, '18'], [7.796296296296297, '03']]


[[38.5948275862069, '15'],
 [23.810344827586206, '02'],
 [21.525, '20'],
 [16.796296296296298, '16'],
 [16.009174311926607, '21'],
 [14.741176470588234, '13'],
 [13.440677966101696, '10'],
 [13.233644859813085, '14'],
 [13.20183486238532, '18'],
 [11.46, '17'],
 [11.383333333333333, '01'],
 [11.051724137931034, '11'],
 [10.8, '19'],
 [10.25, '08'],
 [10.08695652173913, '05'],
 [9.41095890410959, '12'],
 [9.022727272727273, '06'],
 [8.127272727272727, '00'],
 [7.985294117647059, '23'],
 [7.852941176470588, '07'],
 [7.796296296296297, '03'],
 [7.170212765957447, '04'],
 [6.746478873239437, '22'],
 [5.5777777777777775, '09']]

In [10]:
print("Top 5 Hours for Ask Posts Comments")
for avg, hr in sorted_swap[:5]:
    print(
        "{}: {:.2f} average comments per post".format(
            dt.datetime.strptime(hr, "%H").strftime("%H:%M"),avg
        )
    )

Top 5 Hours for Ask Posts Comments
15:00: 38.59 average comments per post
02:00: 23.81 average comments per post
20:00: 21.52 average comments per post
16:00: 16.80 average comments per post
21:00: 16.01 average comments per post


**Conclusion**

The hour that receives the most comments per post on average is 15:00, with an average of 38.59 comments per post. There's about a 60% increase in the number of comments between the hours with the highest and second highest average number of comments.

According to the data set documentation, the timezone used is Eastern Time in the US. So for my time zone that would be 23:00.